### Preliminaries

In [2]:
# import libraries
import torch 
import numpy as np
from torch import nn 
from torch import optim
from torchvision import datasets, transforms 
from torch.utils.data import random_split, DataLoader
# Check cuda availbility 
print('CUDA is available!') if  torch.cuda.is_available() else print('No cuda')
# assign cuda to device
device = torch.device('cuda')
import matplotlib.pyplot as plt
# %matplotlib inline

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


CUDA is available!


### Example 2: CNN for CIFAR10
5 main components 
- model 
- load data 
- trainig loop 
- validation loop 
- hparams, optmizer, and loss
- Start the process

#### 1- Build model Arch

In [4]:
# CNN Model definition for CIFCAR10 datset       
class CNN_M1(nn.Module):
    def __init__(self, input_channels, output_size):
        super().__init__()
        # convolutional layer (sees 32x32x3 image tensor if CFAR)
        self.conv1 = nn.Conv2d(input_channels, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, output_size)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

class CNN_M2(nn.Module):
    def __init__(self, input_channels, output_size):
        super().__init__()
        self.cnn = nn.Sequential(
            # convolutional layer (sees 32x32x3 image tensor if CFAR)
            nn.Conv2d(input_channels, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            # convolutional layer (sees 16x16x16 tensor)
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # convolutional layer (sees 8x8x32 tensor)
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            # flatenning
            nn.Flatten(),
            # linear layer (64 * 4 * 4 -> 500)
            nn.Dropout(0.25),
            nn.Linear(64 * 4 * 4, 500),
            nn.ReLU(),
            
            # linear layer (500 -> 10)
            nn.Dropout(0.25),
            nn.Linear(500, output_size),
        )
    def forward(self, x):
        scores = self.cnn(x)
        return scores

#### 2- Load the data

In [5]:
def load_CIFAR10():
    # Data transforms and augmentation
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(), # randomly flip and rotate
        transforms.RandomRotation(10),
        transforms.ToTensor(), # convert PIL image to Tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ]) # normalizing each channel mean and std

    # Load CIFAR10 train data
    train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
  # load test data 
    test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

    # Split train data to train and val
    val_size = 0.2 
    num_val_samples = int(0.2 * len(train_data))
    num_train_samples = len(train_data) - num_val_samples
    train, val = random_split(train_data, [num_train_samples, num_val_samples])

  
    # Pass the data to the dataloader
    train_loader = DataLoader(train, batch_size = 32)
    val_loader = DataLoader(val, batch_size= 32)
    test_loader = DataLoader(test_data, batch_size= 32)
    
    return train_loader, val_loader, test_loader

In [6]:
# loading mnist data
train_dl, val_dl, test_dl = load_CIFAR10()
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


#### 3- Train loop

In [12]:
def model_train(epochs, optimizer, criterion, model):
    # loop through each epoch
    for epoch in range(epochs):
        # losses and accuraces accumulate
        losses =[]
        accuracies =[]
        
        # activate model train
        model.train()
        
        # loop through each batch
        for batch_idx, (data, labels) in enumerate(train_dl):
            # send data and taret to cuda
            data = data.to(device)
            labels = labels.to(device)

            # forward pass: compute predicted outputs by passing inputs to the model
            scores = model(data)

            # compute the loss function 
            loss = criterion(scores, labels)

            # clear the gradients of the optimizer
            optimizer.zero_grad()

            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()

            # perform a single optimization step based on the computed gradients
            optimizer.step()
            # append loss for each batch
            losses.append(loss.detach().item()) # deteach the loss from compute graph 
            accuracies.append(labels.eq(scores.detach().argmax(dim=1)).float().mean())
        
        # print for each epoch
        print(f'Epoch: {epoch + 1} \n \t Train Loss:{torch.tensor(losses).mean():0.2f} \t Train Acc:{torch.tensor(accuracies).mean():0.2f}')

        # for each epoch evaluate your model on the validation data
        val_loss, val_acc = model_evaluate(criterion, model, phase = 'Val')
        print(f'\t Val Loss:{val_loss:0.2f} \t\t Val Acc:{val_acc:0.2f}')
    return model

#### 4- Val loop

In [13]:
def model_evaluate(criterion, model, phase = 'Val'): # phase can be validate or test
    # append losses and accuracies
    losses = []
    accuracies = []
    # activate model eval function
    model.eval()
    
    dataloader = val_dl if phase == 'Val' else test_dl
    # loop through val data loader
    with torch.no_grad():
        for batch_idx, (data, labels) in enumerate(dataloader):
            # send data and taret to cuda
            data = data.to(device)
            labels = labels.to(device)

            # forward pass: compute predicted outputs by passing inputs to the model
            scores = model(data)

            # compute the loss function 
            loss = criterion(scores, labels)

            # append loss for each batch
            losses.append(loss.detach().item()) # deteach the loss from compute graph 
            accuracies.append(labels.eq(scores.detach().argmax(dim=1)).float().mean())
        mean_loss = torch.tensor(losses).mean()
        mean_acc =torch.tensor(accuracies).mean()
        return mean_loss, mean_acc

#### 5- Optimizer and hparams

In [14]:
# contruct model object
num_channels = 3
num_classes = 10
my_cnn = CNN_M2(num_channels,num_classes).to(device)
# Define my optimiser
params = my_cnn.parameters()
# Using adam optimizer
optimizer = optim.SGD(params, lr=0.01, momentum=0.9)

# Define my loss
# here we use cross entropy loss function
criterion = nn.CrossEntropyLoss()

# hparams 
num_epochs = 5

#### Now lets start training CNN !

In [15]:
cnn_trained = model_train(num_epochs, optimizer, criterion, my_cnn)

Epoch: 1 
 	 Train Loss:1.73 	 Train Acc:0.36
	 Val Loss:1.38 		 Val Acc:0.49
Epoch: 2 
 	 Train Loss:1.33 	 Train Acc:0.52
	 Val Loss:1.15 		 Val Acc:0.59
Epoch: 3 
 	 Train Loss:1.17 	 Train Acc:0.58
	 Val Loss:1.05 		 Val Acc:0.62
Epoch: 4 
 	 Train Loss:1.08 	 Train Acc:0.62
	 Val Loss:0.97 		 Val Acc:0.66
Epoch: 5 
 	 Train Loss:1.02 	 Train Acc:0.64
	 Val Loss:0.95 		 Val Acc:0.67


#### Testing onthe test data

In [16]:
test_loss, test_acc = model_evaluate(criterion, cnn_trained, phase='test')
print(f'Test Loss:{test_loss.item():0.2f}')
print(f'Test Acc:{test_acc.item():0.2f}')

Test Loss:0.95
Test Acc:0.66
